In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen 
import json
import numpy
import requests
import re

####################### Define Variables and Functions ########################################

view_url = "https://pin-code.org.in" 
comp_url = "companies/viewall/"
bank_url = "banks/viewall/"
hosp_url = "Hospitals/viewall/"
crawl_url = ''

def mainfun(url,pin_loop,outdf):
    bank = []
    ngo = []
    hospital = []
    row_final =[]
    titlehref = []
    hosp_urls = []
    bank_urls = []
    comp_cnt = 0
    bank_cnt = 0
    ngo_cnt = 0
    hosp_cnt = 0
    hosp_pvt_cnt = 0
    hosp_pub_cnt = 0
    str_cnt =0 
    url_list_visited = []
    url_list_final = []
    
    
######################## Start : this function is to Crawl the pages ##########################################
    def crawl_fun(crawl_url,url_list_final):
        url_list_temp = []

        res = requests.get(crawl_url).content
        soup = BeautifulSoup(res,'html.parser')
        url_class = soup.find_all("ul", {"class" : "pagination pt-3 pt-md-0"})
        for i in url_class:
            b = i.select("a",href = True)
            for c in b:
                url_list_temp.append(view_url+c.get('href'))

        for temp_url in url_list_temp:
            if temp_url not in url_list_final:
                url_list_final.append(temp_url)
        return()  

    def crawl_visited(crawl_url,url_list_visited,url_list_final):
        while len(url_list_visited) < len(url_list_final) or (len(url_list_visited) < 10) :
            for cur_url in url_list_final:
                crawl_fun(cur_url,url_list_final)
                url_list_visited.append(cur_url)
            return()

######################## End : this function is to Crawl the pages ##########################################

######################## Start : this function is split the string and get count #########################################
    def str_split(str1):
        str_cnt1 = ''
        for f1 in str1:
            if f1.isdigit():
                str_cnt1 = str_cnt1 + f1
        str_cnt = int(str_cnt1)
        return(str_cnt)

######################## End : this function is split the string and get count ##########################################

######################## Start - this logic is to scrap and read the summary of counts  #################################

    res = requests.get(url).content
    soup = BeautifulSoup(res,'html.parser')
    
    html_sumry = soup.find_all('h5',class_ = 'text-center')
    for i in html_sumry:
        sumry = i.get_text()
        sumry = str(sumry)
        str1 = sumry.split(":")
        str2 = str1[1].split(",")
        
        for j in str2:
            if 'Banks' in j:
                bank_cnt = str_split(j)
            if 'Companies' in j:
                comp_cnt = str_split(j)  
            #if 'Post Offices' in j:
            #    print("Postoffice count :" + str(str_split(j)))
            if 'Hospitals' in j:
                hosp_cnt = str_split(j)
        if bank_cnt < 11:
            bank_cnt = 0
        #if comp_cnt < 11:
        #    comp_cnt = 0
        if hosp_cnt < 1:
            hosp_cnt = 0      
######################## End - this logic is to Scrap and read the summary of counts  ######################################

######################## Start - this logic is to Scrap the contents of page  ##########################################
           
    html_parts = soup.find_all('div', class_= 'col-sm-6 col-12 pr-sm-2')

    for i in html_parts:
        b = i.select_one("a",href = True)
        check_var = b['href']
        if (check_var.find('https://pin-code.org.in/banks') == 0 ) and (bank_cnt < 11):
            bank.append(str(b.contents[0]))
            bank_cnt = bank_cnt + 1
        if (check_var.find('https://pin-code.org.in/hospitals') == 0 ) and (hosp_cnt < 1) :
            hospital.append(str(b.contents[0]))
            hosp_cnt = hosp_cnt + 1
        if (check_var.find('https://pin-code.org.in/ngo') == 0 ):
            ngo.append(str(b.contents[0]))
            ngo_cnt = ngo_cnt + 1
            
############################ End - this logic is to Scrap the contents of page  ##########################################


######################## Start - Use additional crawling if count is greater than 10 ###############################
       
    if (bank_cnt > 10):
        crawl_url = view_url + "/"+ bank_url + str(pin_loop)
        url_list_visited = []
        url_list_final = []
        url_list_final.append(crawl_url)
        crawl_visited(crawl_url,url_list_visited, url_list_final)
        bank_urls = url_list_final
        
    if (hosp_cnt > 0):
        crawl_url = view_url + "/"+ hosp_url + str(pin_loop)
        url_list_visited = []
        url_list_final = []
        url_list_final.append(crawl_url)
        crawl_visited(crawl_url,url_list_visited, url_list_final)
        hosp_urls = url_list_final
  
 ######################## End - Use additional crawling if count is greater than 10 #####################################
    
    
######################## Start - Use additional crawling if count is greater than 10 ###############################    
        
    if (bank_cnt > 10):
        for crw_url in bank_urls: 
            res = requests.get(crw_url).content
            soup = BeautifulSoup(res,'html.parser')
            url_class = soup.find_all("h3", {"class" : "panel-title"})
            for i in url_class:
                bank.append(str(i.get_text())) 
                
    if (hosp_cnt > 0):
        for crw_url in hosp_urls: 
            res = requests.get(crw_url).content
            soup = BeautifulSoup(res,'html.parser')
            url_class = soup.find_all("div", {"class" : "companyDetails"})
            for i in url_class:
                b = i.select_one("a",href = True)
                check_var = b['href']
                if (check_var.find('https://pin-code.org.in/hospitals/details/') == 0):
                    hospital.append(str(b.contents[0]))
                    hosp_cat = i.contents[7].get_text().split(":")
                    if hosp_cat[1] == 'Private':
                        hosp_pvt_cnt = hosp_pvt_cnt + 1
                    else:
                        hosp_pub_cnt = hosp_pub_cnt + 1
                
                    
                    
  
 ######################## End - Use additional crawling if count is greater than 10 #####################################
    
 ######################## Start - this logic is to build the dataframe from contents collected ###########################
    pin_list = [pin_loop]
    bank_cnt_list = [bank_cnt]
    company_cnt_list = [comp_cnt]
    ngo_cnt_list = [ngo_cnt]
    hospital_cnt_list = [hosp_cnt]
    hosp_pvt_list = [hosp_pvt_cnt]
    hosp_pub_list = [hosp_pub_cnt]
   
   
    hospital_list = []
    hospital_list.append(hospital[0:])
    
    ngo_list = []
    ngo_list.append(ngo[0:])
    
    
    bank_list = []
    bank_list.append(bank[0:])
        
        
    row_final = pin_list + bank_cnt_list + bank_list + company_cnt_list #+ company_list 
    
    row_final = row_final + hospital_cnt_list + hosp_pvt_list + hosp_pub_list + hospital_list +  ngo_cnt_list + ngo_list
    
    dict1 = zip(varlist,row_final)
    dict2 = dict(dict1)
    outdf.loc[len(outdf.index)] = row_final
    return 

 ######################## End of the logic to build the dataframe from contents collected ###########################
  

 ######################## Main Program to loop each pincode from excel  ###########################
#main program
fix_url = "https://pin-code.org.in/pincode/"
data = pd.read_excel('C:/Users/Admin/Desktop/ISB/Term1/DC/assignment/Indian_Pincodes.xlsx')
postlist = (data['pin_code'])
#postlist = [500001]
#outdf = pd.DataFrame()
varlist = ['Postal Code','Bank_Count','BankNames','Company_Count']
#varlist.append('CompanyNames')
varlist.append('Hospital_Count')
varlist.append('PvtHospital_Count')
varlist.append('GovtHospital_Count')
varlist.append('HospitalNames')
varlist.append('NGO_Count')
varlist.append('NGONames')
outdf = pd.DataFrame(columns =varlist)
#print(outdf)
limit = 0
pinloop = 0
for pin_loop in postlist:
    url = fix_url + str(pin_loop)
    print(url + " : Processing")
    limit = limit + 1
    #### change the value below to limit the loop ####
    ##if limit > 100:
     ##   break;
        
    mainfun(url,pin_loop,outdf)
    
#print(outdf)

######################## End of Main Program to loop each pincode from excel  ###########################

######################## Logic to write the dataframe to Json file  ###########################

#### Uncomment below 2 lines to write to json ############
#json_data = outdf.to_json(orient='records')
#print(json_data)

########################  End of Logic to write the dataframe to Json file  ###########################

######################## Logic to write the dataframe to excel  file  ###########################

#### Uncomment below 3 lines to write to excel ############
writer = pd.ExcelWriter('C:/Users/Admin/Desktop/ISB/Term1/DC/assignment/pincode_out1.xlsx', engine='xlsxwriter')
outdf.to_excel(writer,sheet_name = 'Sheet1', index=False)
writer.save() 
#### Uncomment to write to excel ############
#####outdf.to_excel('C:/Users/Admin/Desktop/ISB/Term1/DC/assignment/pincode_out.xlsx', header = True)

######################## End of Logic to write the dataframe to excel  file  ###########################

https://pin-code.org.in/pincode/224207 : Processing
https://pin-code.org.in/pincode/244715 : Processing
https://pin-code.org.in/pincode/246700 : Processing
https://pin-code.org.in/pincode/271904 : Processing
https://pin-code.org.in/pincode/272206 : Processing
https://pin-code.org.in/pincode/281406 : Processing
https://pin-code.org.in/pincode/284135 : Processing
https://pin-code.org.in/pincode/303503 : Processing
https://pin-code.org.in/pincode/305009 : Processing
https://pin-code.org.in/pincode/305601 : Processing
https://pin-code.org.in/pincode/305624 : Processing
https://pin-code.org.in/pincode/306305 : Processing
https://pin-code.org.in/pincode/313015 : Processing
https://pin-code.org.in/pincode/333042 : Processing
https://pin-code.org.in/pincode/341012 : Processing
https://pin-code.org.in/pincode/363250 : Processing
https://pin-code.org.in/pincode/364070 : Processing
https://pin-code.org.in/pincode/380045 : Processing
https://pin-code.org.in/pincode/380052 : Processing
https://pin-